In [2]:
import mysql.connector
from mysql.connector import Error

# Preliminary note

You can skip this notebook and instead download the WikiGazetteers required for the experiments directly from [here]**(ADD LINK TO WIKIGAZETTEERS IN ZENODO)**.

# Processing WikiGazetteer

WikiGazetteer is a gazetteer based on Wikipedia and enriched with Geonames data. To build a WikiGazetteer into a MySQL database for a specific Wikipedia language and version, follow [these instructions](https://github.com/Living-with-machines/lwm_GIR19_resolving_places/tree/master/gazetteer_construction). This notebook assumes the user already has created WikiGazetteers for English, Spanish, and Greek, which are stored in a MySQL database.

### Create minimal Wikigazetteer

The following cells create a minimal WikiGazetteer. It results in a dataframe (which we store in `processing/resources/` as a pickle file), where each row corresponds to an altname-location pair. The dataframe has the following fields:
* **altname:** alternate name of a location.
* **pid:** persistent identifier of the location (e.g. the wikipedia title in WikiGazetteer).
* **lat:** latitude of the location.
* **lon:** longitude of the location.
* **source:** source of the alternate name (geonames: 'geonamesalt', 'geonamesmain', 'geonamesascii'; or wikipedia: 'wikimain', 'wikiredirect').

In [2]:
def create_minimal_gaz(gazetteer):
    gazDB = ""
    cursorGaz = ""
    try:
        gazDB = mysql.connector.connect(
                host='localhost',
                database=gazetteer,
                user='testGazetteer',
                password='1234')
        if gazDB.is_connected():
            cursorGaz = gazDB.cursor(buffered=True)
    except Error as e:
        print("Error while connecting to MySQL", e)
        
    cursorGaz.execute("""SELECT altname, wiki_title, lat, lon, source FROM altname
                         JOIN location ON altname.main_id = location.id""")
    locs = cursorGaz.fetchall()
    df = pd.DataFrame(locs, columns =['altname', 'pid', 'lat', 'lon', 'source'])
    
    # Close DB connection:
    if (gazDB.is_connected()):
        cursorGaz.close()
        gazDB.close()
    
    gaznames = {"wikiGazES":"wikigaz_es",
            "wikiGazEL":"wikigaz_el",
            "wikiGazEN": "wikigaz_en"}
    
    df.to_pickle("../resources/" + gaznames[gazetteer] + "_basic.pkl")

In [4]:
# Create minimal gazetteer: the argument is the name of the MySQL database
create_minimal_gaz("wikiGazES")
create_minimal_gaz("wikiGazEL")
create_minimal_gaz("wikiGazEN")